**Imports**

In [12]:
import numpy as np
import pandas as pd
import os 

import tensorflow as tf

from sbb import Sbb

%matplotlib inline
import matplotlib.pyplot as plt

**Global variables**

In [13]:
model_folder = '../model'

checkpoint_path = model_folder + "/sbb.ckpt"

output_size = 7
input_size = (1, 4)

In [14]:
if not os.path.exists(model_folder):
    os.mkdir(model_folder)

In [15]:
physical_devices = tf.config.list_physical_devices("GPU")

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

**Retrive data from system**

In [16]:
data = pd.read_csv('../data/output.csv').dropna()
train_count = round(len(data) * 0.8)

train_data, validation_data = data.iloc[:train_count], data.iloc[train_count:]

In [48]:
def fire(raw_data, batch_size=128):
    return_index = 0
    max_index = len(raw_data) // batch_size
    
    for i in range(1, max_index):
        t_data = raw_data[batch_size * i:batch_size * (i+1)]
        return t_data['binary_hours'].tolist()
        #yield np.expand_dims(t_data.to_numpy()[:, :4], axis=1), np.expand_dims(t_data.to_numpy()[:, 4:], axis=1)

In [49]:
fire(train_data)

['[0, 0, 0, 0, 0, 0, 0]',
 '[0, 1, 1, 0, 1, 1, 1]',
 '[0, 1, 1, 0, 1, 1, 1]',
 '[0, 0, 0, 0, 1, 1, 0]',
 '[0, 0, 0, 0, 0, 0, 0]',
 '[0, 0, 0, 0, 0, 0, 0]',
 '[0, 0, 1, 1, 1, 1, 1]',
 '[1, 1, 1, 1, 1, 1, 0]',
 '[1, 1, 1, 1, 1, 1, 1]',
 '[1, 1, 1, 1, 1, 1, 1]',
 '[0, 0, 0, 0, 0, 0, 0]',
 '[0, 0, 0, 0, 0, 1, 1]',
 '[1, 1, 0, 1, 0, 1, 0]',
 '[1, 1, 0, 1, 0, 1, 0]',
 '[1, 1, 0, 1, 0, 0, 0]',
 '[0, 0, 0, 1, 1, 1, 0]',
 '[1, 1, 1, 0, 1, 0, 0]',
 '[1, 1, 0, 1, 1, 0, 1]',
 '[1, 1, 0, 1, 1, 0, 1]',
 '[1, 0, 1, 0, 1, 0, 1]',
 '[0, 0, 0, 1, 1, 1, 0]',
 '[0, 0, 0, 1, 0, 1, 1]',
 '[0, 0, 0, 1, 0, 0, 1]',
 '[0, 0, 0, 0, 0, 0, 0]',
 '[0, 1, 1, 0, 0, 0, 0]',
 '[0, 0, 1, 0, 1, 1, 1]',
 '[0, 0, 1, 0, 0, 0, 0]',
 '[0, 0, 1, 0, 0, 0, 0]',
 '[0, 0, 0, 1, 1, 0, 0]',
 '[0, 0, 0, 0, 0, 0, 0]',
 '[0, 0, 0, 0, 0, 0, 0]',
 '[1, 1, 1, 1, 1, 1, 1]',
 '[0, 1, 1, 1, 0, 1, 0]',
 '[0, 0, 0, 0, 0, 0, 0]',
 '[1, 1, 1, 1, 1, 1, 1]',
 '[1, 1, 1, 1, 1, 1, 1]',
 '[1, 1, 1, 1, 1, 1, 1]',
 '[1, 0, 0, 0, 0, 0, 0]',
 '[0, 0, 1, 

In [20]:
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(raw_data=train_data),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1, output_size]))

test_dataset = tf.data.Dataset.from_generator(
    generator=lambda: fire(raw_data=validation_data),
    output_types=(tf.float64, tf.float64),
    output_shapes=([None] + list(input_size), [None, 1, output_size]))

In [21]:
a,b = next(iter(train_dataset))
a.shape, b.shape

InvalidArgumentError: TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was float64, but the yielded element was [[['[0, 0, 0, 0, 0, 0, 0]']]

 [['[0, 1, 1, 0, 1, 1, 1]']]

 [['[0, 1, 1, 0, 1, 1, 1]']]

 [['[0, 0, 0, 0, 1, 1, 0]']]

 [['[0, 0, 0, 0, 0, 0, 0]']]

 [['[0, 0, 0, 0, 0, 0, 0]']]

 [['[0, 0, 1, 1, 1, 1, 1]']]

 [['[1, 1, 1, 1, 1, 1, 0]']]

 [['[1, 1, 1, 1, 1, 1, 1]']]

 [['[1, 1, 1, 1, 1, 1, 1]']]

 [['[0, 0, 0, 0, 0, 0, 0]']]

 [['[0, 0, 0, 0, 0, 1, 1]']]

 [['[1, 1, 0, 1, 0, 1, 0]']]

 [['[1, 1, 0, 1, 0, 1, 0]']]

 [['[1, 1, 0, 1, 0, 0, 0]']]

 [['[0, 0, 0, 1, 1, 1, 0]']]

 [['[1, 1, 1, 0, 1, 0, 0]']]

 [['[1, 1, 0, 1, 1, 0, 1]']]

 [['[1, 1, 0, 1, 1, 0, 1]']]

 [['[1, 0, 1, 0, 1, 0, 1]']]

 [['[0, 0, 0, 1, 1, 1, 0]']]

 [['[0, 0, 0, 1, 0, 1, 1]']]

 [['[0, 0, 0, 1, 0, 0, 1]']]

 [['[0, 0, 0, 0, 0, 0, 0]']]

 [['[0, 1, 1, 0, 0, 0, 0]']]

 [['[0, 0, 1, 0, 1, 1, 1]']]

 [['[0, 0, 1, 0, 0, 0, 0]']]

 [['[0, 0, 1, 0, 

In [10]:
sbb = Sbb(checkpoint_path, output_size, input_size)

In [11]:
sbb.train(train_dataset, test_dataset, epochs=2000, batch_size=128)

(1, 4)
Model: "DENSE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 4)]            0         
_________________________________________________________________
DENSE-1-DENSE-1 (Dense)      (None, 1, 256)            1280      
_________________________________________________________________
DENSE-1-DENSE-2 (Dense)      (None, 1, 256)            65792     
_________________________________________________________________
DENSE-1-DENSE-3 (Dense)      (None, 1, 256)            65792     
_________________________________________________________________
dense (Dense)                (None, 1, 1)              257       
Total params: 133,121
Trainable params: 133,121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
29/29 [==============================] - 1s 10ms/step - loss: 44116.0312 - mean_squared_error: 44116.

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_dataset)